<a href="https://colab.research.google.com/github/gi3265/-Kaggle-Disaster_Tweets/blob/hyebin/yj_pre%26fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.전처리

In [1]:
# 데이터 전처리 -> tweet-preprocessor 이용
!pip install tweet-preprocessor

In [1]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import abc
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import string
import re
from nltk.corpus import wordnet as wn
import preprocessor 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn import metrics

In [3]:
import os

os.chdir('/content/drive/MyDrive/캐글트위터')
os.getcwd()

'/content/drive/MyDrive/캐글트위터'

In [4]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [20]:
stop = set(STOPWORDS).union(set(['FAV' , 'RT']))  ## .union() =  합집합
lemma = WordNetLemmatizer()  ## NLTK에서는 표제어(단어의 뿌리) 추출을 위한 도구인 WordNetLemmatizer를 지원
preprocessor.set_options(preprocessor.OPT.URL, preprocessor.OPT.MENTION, preprocessor.OPT.NUMBER, preprocessor.OPT.RESERVED)  ## OPT = option

def clean(text):   
    text = preprocessor.clean(text)
    text = re.sub(r'[^\w\s]','',text)
    stop_free = " ".join([i for i in text.split(' ') if (i not in stop)])
    normalized = " ".join(lemma.lemmatize(word) for word in stop_free.split())
    return normalized.lower()

In [21]:
train['text2'] = train['text'].apply(clean)

In [22]:
train.head()

,id,keyword,location,text,target,text2,text3
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds reason earthquake may allah forgive u,__label__1 Our Deeds Reason earthquake May A...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,__label__1 Forest fire near La Ronge Sask Ca...
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all resident asked shelter place notified offi...,__label__1 All resident asked shelter place ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...,__label__1 people receive wildfire evacuatio...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent photo ruby alaska smoke wildfire...,__label__1 Just got sent photo Ruby Alaska s...


# 2. fasttext 이용하기 위해 데이터 정제

In [23]:
for i in range(train.shape[0]):
    train.loc[i, 'text3'] = '__label__' + str(train.loc[i, 'target']) + '   ' + train.loc[i, 'text2']

train.head()

,id,keyword,location,text,target,text2,text3
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds reason earthquake may allah forgive u,__label__1 our deeds reason earthquake may a...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,__label__1 forest fire near la ronge sask ca...
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all resident asked shelter place notified offi...,__label__1 all resident asked shelter place ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...,__label__1 people receive wildfire evacuatio...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent photo ruby alaska smoke wildfire...,__label__1 just got sent photo ruby alaska s...


In [24]:
train.text3.to_csv('./fasttext_label.txt', index = False, header = False)

In [12]:
pip install fasttext

     |████████████████████████████████| 71kB 7.3MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3097175 sha256=4f24bf0b3a576a75dcd218b84a9b6cb207df5e07b8cc3d1a58a9038f26827f21
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


# 3. fasttext 활용

In [14]:
import fasttext

In [25]:
classifier = fasttext.train_supervised('fasttext_label.txt', label_prefix='__label__')

In [26]:
test['text2'] = test['text'].apply(clean)

In [27]:
test.head()

,id,keyword,location,text,text2
0,0,NaN,NaN,Just happened a terrible car crash,just happened terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard earthquake different city stay safe ever...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",forest fire spot pond goose fleeing across str...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfire
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kill china taiwan


In [32]:
sub = pd.read_csv('./sample_submission.csv')
sub.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [51]:
texts = []

for i in range(test.shape[0]):
    texts.append(test.iloc[i, -1])

texts[:10]

['just happened terrible car crash',
 'heard earthquake different city stay safe everyone',
 'forest fire spot pond goose fleeing across street i save',
 'apocalypse lighting spokane wildfire',
 'typhoon soudelor kill china taiwan',
 'were shakingits earthquake',
 'theyd probably still show life arsenal yesterday eh eh',
 'hey how',
 'what nice hat',
 'fuck']

In [52]:
labels = classifier.predict(texts)
print (labels)

([['__label__1'], ['__label__1'], ['__label__1'], ['__label__1'], ['__label__1'], ['__label__1'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__1'], ['__label__0'], ['__label__1'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__1'], ['__label__0'], ['__label__1'], ['__label__0'], ['__label__1'], ['__label__0'], ['__label__1'], ['__label__0'], ['__label__0'], ['__label__1'], ['__label__0'], ['__label__1'], ['__label__0'], ['__label__1'], ['__label__0'], ['__label__0'], ['__label__1'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__1'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__0'], ['__label__1'], ['__label__0'], ['__la

# 4. submission에 예측값 채워주기

In [58]:
for i in range(len(texts)):
    sub.iloc[i, -1] = int(labels[0][i][0][-1])

In [59]:
sub

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [61]:
sub.to_csv('./final_sample.csv', index=False, header=True) # 0.79068 기록